# Heapmod Algorithm For Computing The Minimum Free Distance Of Convolutional Codes
### by Eng. O. David and Dr. V. Lyandres

In [1]:
def hamming_weight(matrix):
    weight = 0
    for i in range(matrix.nrows()):
        for j in range(matrix.ncols()):
            weight += int(matrix[i, j])
    return weight

def dec_to_bin(number, length):
    # computes the binary representation of a decimal number in the way we need it
    # for the registers
    str_list = list(bin(number))[2:]
    bin_list = [0 for _ in range(length-len(str_list))] + [int(s) for s in str_list]
    bin_list.reverse()
    return matrix(1, length, bin_list)

def calculate_sequence(l_j, memory_length):
    # calculates the dividing by 2 and floor sequence for the l_j.
    # i.e. calculates the heap path
    a = l_j
    sequence = [a%(2^memory_length)]
    while a != 1:
        a = a // 2
        sequence.append(a%(2^memory_length))
    return sequence

def weight_of_sequence(sequence, weights):
    # adds the weights of the nodes of the sequence.
    weight = 0
    for i in range(len(sequence)):
        weight += weights[sequence[i]]
    return weight
    
def coefficients_up_to_deg(f, n):
    # returns the coefficients of a polynomial f up to the coefficient of degree n.
    coeff_list = f.coefficients(sparse=False)
    coeff_list = coeff_list + [0 for _ in range(n - len(coeff_list))]
    return matrix(GF(2), n, 1, coeff_list)

def heapmod(generator_matrix):
    polynomials = generator_matrix[0]
    n = len(polynomials)
    memory_length = max([polynomial.degree() for polynomial in polynomials]) + 1
    # make generator polynomial matrix into the g_i
    g_i_list = [coefficients_up_to_deg(f, memory_length) for f in polynomials]
    # generate the matrix to get the outputs for each node.
    G_M_n = block_matrix(1, n, g_i_list, subdivide=False)
    registers = [dec_to_bin(i, memory_length) for i in range(2^memory_length)]
    outputs = [registers[i] * G_M_n for i in range(2^memory_length)]
    weights = [hamming_weight(outputs[i]) for i in range(2^memory_length)]
    
    L_j = [(2*j + 1) * 2^(memory_length-1) for j in range(2^memory_length)]
    sequences = [calculate_sequence(l_j, memory_length) for l_j in L_j]
    
    # calculate the weight/distance for each path corresponding to the l_j's.
    weights = [weight_of_sequence(sequences[i], weights) for i in range(len(sequences))]
    return min(weights)

In [3]:
# counterexample for the heapmod.
R.<x> = PolynomialRing(GF(2), 'x')
generator_matrix = matrix(R, [[x^6 + x^4 + 1, x^6 + x^5 + x^4 + x^3 + x + 1]])
print(heapmod(generator_matrix))

9
